### 1. 训练自定义模型。

In [1]:
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

def my_input_fn(file_path, perform_shuffle=False, repeat_count=1):
    def decode_csv(line):
        parsed_line = tf.decode_csv(line, [[0.], [0.], [0.], [0.], [0]])
        return {"x": parsed_line[:-1]}, parsed_line[-1:]

    dataset = (tf.contrib.data.TextLineDataset(file_path)  
               .skip(1)  
               .map(decode_csv))  
    if perform_shuffle:
        dataset = dataset.shuffle(buffer_size=256)
    dataset = dataset.repeat(repeat_count) 
    dataset = dataset.batch(32) 
    iterator = dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    return batch_features, batch_labels

feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,  
    hidden_units=[10, 10],  
    n_classes=3)

classifier.train(
    input_fn=lambda: my_input_fn("../../datasets/iris_training.csv", True, 100))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11e1081d0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/v9/vrbrlhtd1dz8tdzl3s3d4d3c0000gn/T/tmpZweRVe', '_save_summary_steps': 100}
Instructions for updating:
Use `tf.data.TextLineDataset`.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/v9/vrbrlhtd1dz8tdzl3s3d4d3c0000gn/T/tmpZweRVe/model.ckpt.
INFO:tensorflow:loss = 64.0272, step = 1
INFO:tensorflow:global_step/sec: 119.28
INFO:tensorflow:loss = 6.56172, step = 101 (0.867 sec)
INFO:tensorflow:global_step

### 2. 测试模型。

In [2]:
test_results = classifier.evaluate(
    input_fn=lambda: my_input_fn("../../datasets/iris_test.csv", False, 1))
print("\nTest accuracy: %g %%" % (test_results["accuracy"]*100))

INFO:tensorflow:Starting evaluation at 2017-12-25-22:12:36
INFO:tensorflow:Restoring parameters from /var/folders/v9/vrbrlhtd1dz8tdzl3s3d4d3c0000gn/T/tmpZweRVe/model.ckpt-375
INFO:tensorflow:Finished evaluation at 2017-12-25-22:12:42
INFO:tensorflow:Saving dict for global step 375: accuracy = 0.966667, average_loss = 0.102634, global_step = 375, loss = 3.07902

Test accuracy: 96.6667 %
